In [8]:
import numpy as np

# 定义线性弹性模型
def linear_elastic_model(params, strain):
    E = params['E']
    nu = params['nu']
    stress = E / (1 + nu) * (strain - nu/(1-nu) * np.mean(strain, axis=0))
    return stress

# 定义MCMC模型
def mcmc_model(strain, stress_obs, draws=10000):
    # 定义参数先验分布
    E_mean = 1000
    E_sd = 100
    nu_mean = 0.4
    nu_sd = 0.05
    
    # 初始化参数
    E = np.random.normal(E_mean, E_sd)
    nu = np.random.normal(nu_mean, nu_sd)
    
    # 初始化MCMC过程
    trace = np.zeros((draws, 2))
    accepted = 0
    
    # 运行MCMC
    for i in range(draws):
        # 计算当前参数下的应力
        stress = linear_elastic_model({'E': E, 'nu': nu}, strain)
        
        # 计算似然函数值
        llh_current = -0.5 * np.sum((stress - stress_obs)**2)
        
        # 从先验分布中随机抽取新的参数
        E_new = np.random.normal(E, 10)
        nu_new = np.random.normal(nu, 0.01)
        
        # 计算新参数下的应力
        stress_new = linear_elastic_model({'E': E_new, 'nu': nu_new}, strain)
        
        # 计算新参数下的似然函数值
        llh_new = -0.5 * np.sum((stress_new - stress_obs)**2)
        
        # 计算接受率
        p_accept = np.exp(llh_new - llh_current)
        
        # 接受新参数
        if p_accept > np.random.uniform():
            E = E_new
            nu = nu_new
            accepted += 1
        
        # 保存参数
        trace[i, 0] = E
        trace[i, 1] = nu
    
    # 输出结果
    print(f"Acceptance rate: {accepted/draws:.2f}")
    return trace

# 构建数据
strain = np.array([[0.1, 0.2, 0.3], [0.2, 0.3, 0.4], [0.3, 0.4, 0.5]])
stress_obs = np.array([[10, 20, 30], [20, 30, 40], [30, 40, 50]])

# 运行MCMC
trace = mcmc_model(strain, stress_obs)


/var/folders/y5/_38246qd6zd0lmzngc4zlt8h0000gn/T/ipykernel_59941/3596381980.py:45: RuntimeWarning: overflow encountered in exp
  p_accept = np.exp(llh_new - llh_current)


Acceptance rate: 0.13


这段代码实现了一个使用MCMC（马尔可夫链蒙特卡罗）方法计算线性弹性模型软组织概率参数的示例。

首先，定义了一个线性弹性模型函数linear_elastic_model，该函数接受一个参数字典params和应变（strain）数组，返回应力（stress）数组。这里假设线性弹性模型的杨氏模量（E）和泊松比（nu）是参数字典params中的两个键。具体计算公式可以根据具体情况进行修改，这里给出的是一种常见的计算公式。

然后，定义了一个MCMC模型函数mcmc_model，该函数接受应变（strain）数组和观测应力（stress_obs）数组，并使用MCMC方法计算线性弹性模型的杨氏模量和泊松比的概率分布。该函数首先定义了杨氏模量和泊松比的先验分布，然后初始化参数E和nu，以及MCMC过程需要用到的变量trace和accepted。

在MCMC过程中，首先计算当前参数下的应力（stress）和似然函数值（llh_current），然后从杨氏模量和泊松比的先验分布中抽取新的参数E_new和nu_new，计算新参数下的应力（stress_new）和似然函数值（llh_new），然后计算接受率（p_accept）并根据接受率决定是否接受新参数。最后，将参数E和nu保存到trace数组中，并统计接受率。

在代码末尾，构建了一个应变（strain）数组和观测应力（stress_obs）数组，并调用mcmc_model函数运行MCMC过程，得到杨氏模量和泊松比的概率分布。在实际使用中，可能需要对结果进行进一步的分析和可视化。